In [1]:
import os

import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F

from preprocessing_img_for_segmentation import process
from torch.utils.data import DataLoader, Dataset
import volumentations as V


def create_fold_from_data(path_data, fold=5):
    """
    Проблема в том, что одному patient_id могут относиться 2 изображения.
    Мне нужно разбить дату так, чтобы 2 изображения, которые относятся к одному patient_id
    были в одном фолде.
    :param path_data:
    :param randome_state:
    :param fold:
    :return:
    """
    df = pd.read_csv(path_data)
    df['count'] = df.groupby('patient_id')['patient_id'].transform('count')
    df = df.sort_values(by='count', ascending=False)
    df['FOLD'] = (df.index // (len(df) / fold)).astype(int)

    return df


class SegmentationDataset(Dataset):
    def __init__(self,
                 data,
                 size,
                 path_to_folder_with_imgs,
                 path_to_folder_with_masks,
                 typ_augm,
                 device):

        self.data = data
        self.size = size
        self.path_to_folder_with_imgs = path_to_folder_with_imgs
        self.path_to_folder_with_masks = path_to_folder_with_masks
        self.device = device

        if typ_augm == 'v1':

            self.aug = V.Compose([
                V.Resize(self.size, interpolation=3, resize_type=0, always_apply=True, p=1.0),
            ], p=1.0)



    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        path_to_img = os.path.join(self.path_to_folder_with_imgs, str(row['patient_id']), str(row['series_id']))
        path_to_mask = os.path.join(self.path_to_folder_with_masks, f"{row['series_id']}.npy")

        img = process(data_path=path_to_img)

        img_tensor = F.interpolate(
            torch.tensor(img).unsqueeze(0).unsqueeze(0).to(self.device),
            size=self.size,
            mode='trilinear',
            align_corners=False
        )[0]

        mask = np.load(path_to_mask)
        upd_mask = self.aug(**{"mask":mask})['mask']



        return img_tensor, torch.Tensor(upd_mask)




    

In [2]:
import os
import pydicom
import numpy as np

import ipyvolume as ipv
import volumentations as V
import nibabel as nib
import sys

In [3]:
df = create_fold_from_data(path_data="../data_for_segmentation.csv",
                               )

dataset = SegmentationDataset(data=df,
                      size=(192, 192, 192),
                      typ_augm='v1',
                      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                      path_to_folder_with_imgs=r"C:\KAGGLE\MEDICINE\data\train_images",
                      path_to_folder_with_masks=r"C:\KAGGLE\MEDICINE\data\segmentations_numpy")



In [18]:
img, mask = dataset[29]
print(img.shape, mask.shape)

torch.Size([1, 192, 192, 192]) torch.Size([192, 192, 192])


In [19]:
img = img[0].to('cpu').numpy()

In [20]:
mask = mask.to('cpu').numpy()

In [24]:
fig = ipv.figure()


vol = ipv.volshow(   np.where(mask == 4, 1, 0) )

ipv.style.box_off()
ipv.style.axes_off()
ipv.style.set_style_light()


ipv.show()

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…